# Project 2: Covid ---> III/ Models and predictions

The purpose of this file is to test and compare several models on the matrix extracted from the "II_features-selection" file. We will use a sample of the data for questions of run time. Then the best model will be applied to all the data.

In [79]:
# Import
%matplotlib inline

import os
import os.path as op
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, average_precision_score, auc

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import xgboost as xgb

from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Literature

In 2019, the first COVID-19 cases are observed in China. Rapidly, the SARS-Cov2 virus spread worldwide, pushing governments to take strict decisions about the lives of their co-citizens, like containment, to protect the population. Indeed, in some cases, COVID-19 patients ended up in intensive care services and sometimes died.

**The aim of our model is, based on easily computable parameters at the study's beginning, to predict whether the patient will be likely to die or if the chance of survival is important.** The point of this study is to help the hospital organise in the case of a high number of cases.


The studied dataset stem from the IDDO Data Repository of COVID-19 data. This data was pulled from the underlying data collection projects on 2022-09-01. The data comes from 1,200 institutions from over 45 countries and gather various information from 700,000 hospitalised individuals.

To keep only the relevant features, we first dive into the literature, using Meta-analysis papers. First, we have been looking for aggravating factors that will likely lead the patient to ICU.

Obesity: according to a meta-analysis by Sales-Peres, there is a correlation between obesity and ICU admission. This paper also concluded that co-morbidities for obese patients, such as hypertension, type 2 diabetes, smoking habit, lung disease, and/or cardiovascular disease lead to a higher chance of ICU admission.
Age: patients aged 70 years and above have a higher risk of infection and a higher need for intensive care than patients younger than 70.
Sex: men, when infected, have a higher risk of severe COVID-19 disease and a higher need for intensive care than women\cite{pijls_demographic_2021}.
Ethnicity: the risk of contamination was higher in most ethnic minority groups than their White counterparts in North America and Europe. Among people with confirmed infection, African-Americans and Hispanic Americans were also more likely than White Americans to be hospitalised with SARS-CoV-2 infection. However, the probability of ICU admission was equivalent for all groups. Thus, ethnicity is not relevant to our question. 
Blood tests: Patients with increased pancreatic enzymes, including elevated serum lipase or amylase of either type, had worse clinical outcomes. Lower levels of lymphocytes and hemoglobin; elevated levels of leukocytes, aspartate aminotransferase, alanine aminotransferase, blood creatinine, blood urea nitrogen, high-sensitivity troponin, creatine kinase, high-sensitivity C-reactive protein, interleukin 6, D-dimer, ferritin, lactate dehydrogenase, and procalcitonin; and a high erythrocyte sedimentation rate were also associated with severe COVID-19.  

Out of a total of 3009 citations, 17 articles (22 studies, 21 from China and one study from Singapore) with 3396 ranging from 12 to1099 patients were included. Our meta-analyses showed a significant decrease in lymphocyte, monocyte, and eosinophil, hemoglobin, platelet, albumin, serum sodium, lymphocyte to C-reactive protein ratio (LCR), leukocyte to C-reactive protein ratio (LeCR), leukocyte to IL-6 ratio (LeIR), and an increase in the neutrophil, alanine aminotransferase (ALT), aspartate aminotransferase (AST), total bilirubin, blood urea nitrogen (BUN), creatinine (Cr), erythrocyte Sedimentation Rate (ESR), C-reactive protein (CRP), Procalcitonin (PCT), lactate dehydrogenase (LDH), fibrinogen, prothrombin time (PT), D-dimer, glucose level, and neutrophil to lymphocyte ratio (NLR) in the severe group compared with the non-severe group. 

No significant changes in white blood cells (WBC), Creatine Kinase (CK), troponin I, myoglobin, IL-6 and K between the two groups were observed. 

## 2. Load data after data_selection and feature_selection

The file we open already has: 
- lines with NA for DSDECOD that have been removed
- the NAs that have been filled in 
- the standardisation that has been performed 
- the features we want to keep that have been selected
- the data has been stratified over the continents, and we only kept the data from Europe

In [80]:
# Open file
data_folder = op.join(os.getcwd(), "data", "results")
mylist = []
for chunk in pd.read_csv(op.join(data_folder, 'df_final_II-FeaturesSelection_train_svm.csv'), sep=',', low_memory=False, chunksize=5000, index_col=0):
    mylist.append(chunk)
df_train = pd.concat(mylist, axis=0)
df_train.name = 'df_train'
del mylist

In [81]:
data_folder = op.join(os.getcwd(), "data", "results")
mylist = []
for chunk in pd.read_csv(op.join(data_folder, 'df_final_II-FeaturesSelection_test_svm.csv'), sep=',', low_memory=False, chunksize=5000, index_col=0):
    mylist.append(chunk)
df_test = pd.concat(mylist, axis=0)
df_test.name = 'df_test'
del mylist

In [82]:
df_train.head(3)

,HODECOD,IETEST_Acute_Respiratory_Infection,INCLAS_VACCINES,SACAT_COMORBIDITIES,SACAT_COMPLICATIONS,SACAT_PREVIOUS_COVID-19_INFECTION,MBTEST_OTHER RESPIRATORY PATHOGENS,AGE,LBTEST_APTT,LBTEST_AST,...,LBTEST_HCT,LBTEST_HGB,LBTEST_INR,LBTEST_LACTICAC,LBTEST_LDH,LBTEST_PCT,LBTEST_PT,VSTEST_CPLRFLT,VSTEST_DIABP,DSDECOD
0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.160639,-0.079327,-0.058478,...,0.151193,-0.297221,-0.094140,-0.087076,-0.077041,-0.091185,-0.154999,0.0,-1.410237,0.0
1,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.648660,-0.079327,-0.058478,...,0.151193,-0.038149,-0.094140,-0.100020,-0.077041,-0.091185,-0.154999,0.0,1.241241,0.0
2,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.648660,-0.079327,0.104826,...,-0.626112,-0.027354,-0.082164,-0.096424,-0.077041,-0.093052,-0.154999,0.0,-0.390437,0.0


In [83]:
df_train.shape

(140, 21)

In [84]:
df_train.columns

Index(['HODECOD', 'IETEST_Acute_Respiratory_Infection', 'INCLAS_VACCINES',
       'SACAT_COMORBIDITIES', 'SACAT_COMPLICATIONS',
       'SACAT_PREVIOUS_COVID-19_INFECTION',
       'MBTEST_OTHER RESPIRATORY PATHOGENS', 'AGE', 'LBTEST_APTT',
       'LBTEST_AST', 'LBTEST_BILI', 'LBTEST_HCT', 'LBTEST_HGB', 'LBTEST_INR',
       'LBTEST_LACTICAC', 'LBTEST_LDH', 'LBTEST_PCT', 'LBTEST_PT',
       'VSTEST_CPLRFLT', 'VSTEST_DIABP', 'DSDECOD'],
      dtype='object')

## 3. Search the best model/algorithm and the best parameters

We will work only on a sample of the data to try to find the best algorithm/model with the best parameters. Then we will apply the best model to the whole data set.

In [85]:
# Separate data into features and label
X_train = df_train.loc[:, df_train.columns!='DSDECOD']
y_train = df_train['DSDECOD']

X_test = df_test.loc[:, df_test.columns!='DSDECOD']
y_test = df_test['DSDECOD']

In [11]:
# For storage of results for each model
df_results = pd.DataFrame()

### 3.1. Model 1: Logistic regression

In [9]:
# Choose parameter values to test for cross-validation
param_grid = {"C":np.logspace(-3,3,4), 
              "penalty":["l2"]}

# Choose the estimator
estimator = LogisticRegression(max_iter=200)

# Cross-validation with GridSearchCV
clf = GridSearchCV(estimator, param_grid, scoring='f1', cv=StratifiedKFold(n_splits=3, shuffle=True))

# Fit data into the model
clf.fit(X_train, y_train)

# Predicting values
y_pred = clf.predict(X_test)

# Proba for the greater label
y_score = clf.predict_proba(X_test)[:, 1]

# Best parameters
best_param = clf.best_estimator_.get_params()
parameters = {'C':best_param['C'], 'penalty':best_param['penalty']}
print('Best parameters for Logistic regression:')
print('C:', parameters['C'])
print('Penalty:', parameters['penalty'])
print('------------')

# Calculate performance scores
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_score)
average_precision = average_precision_score(y_test, y_score)

# Print performance
print('Performance for Logistic regression:')
print('  - Accuracy score = {:.2f}'.format(accuracy))
print('  - F1 score = {:.2f}'.format(f1))
print('  - Precision score = {:.2f}'.format(precision))
print('  - Recall score = {:.2f}'.format(recall))
print('  - ROC AUC score = {:.2f}'.format(roc_auc))
print('  - Average precision score = {:.2f}'.format(average_precision))

# Add performance to df_results
df_results = df_results.append(pd.Series({"Model" : "Logistic regression",
                                          "Parameters": parameters,
                                          "Accuracy" : accuracy,
                                          "F1" : f1,
                                          "Precision": precision,
                                          "Recall" : recall,
                                          "ROC AUC" : roc_auc,
                                          "Average precision" : average_precision}), ignore_index=True)

Best parameters for Logistic regression:
C: 1000.0
Penalty: l2
------------
Performance for Logistic regression:
  - Accuracy score = 0.71
  - F1 score = 0.72
  - Precision score = 0.70
  - Recall score = 0.74
  - ROC AUC score = 0.79
  - Average precision score = 0.76


### 3.2. Model 2 : K-Nearest Neighbor Classifier

In [10]:
# Choose parameter values to test for cross-validation
k_range = list(range(5, 21, 5))
param_grid = dict(n_neighbors=k_range)

# Choose the estimator
estimator = KNeighborsClassifier()

# Cross-validation with GridSearchCV
clf = GridSearchCV(estimator, param_grid, scoring='f1', cv=StratifiedKFold(n_splits=3, shuffle=True))

# Fit data into the model
clf.fit(X_train, y_train)

# Predicting values
y_pred = clf.predict(X_test)

# Proba for the greater label
y_score = clf.predict_proba(X_test)[:, 1]

# Best parameters
best_param = clf.best_estimator_.get_params()
parameters = {'k':best_param['n_neighbors']}
print('Best parameters for K-Nearest Neighbor:')
print('k:', parameters['k'])
print('------------')

# Calculate performance scores
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_score)
average_precision = average_precision_score(y_test, y_score)

# Print performance
print('Performance for K-Nearest Neighbor:')
print('  - Accuracy score = {:.2f}'.format(accuracy))
print('  - F1 score = {:.2f}'.format(f1))
print('  - Precision score = {:.2f}'.format(precision))
print('  - Recall score = {:.2f}'.format(recall))
print('  - ROC AUC score = {:.2f}'.format(roc_auc))
print('  - Average precision score = {:.2f}'.format(average_precision))

# Add performance to df_results
df_results = df_results.append(pd.Series({"Model" : "K-Nearest Neighbor",
                                          "Parameters": parameters,
                                          "Accuracy" : accuracy,
                                          "F1" : f1,
                                          "Precision": precision,
                                          "Recall" : recall,
                                          "ROC AUC" : roc_auc,
                                          "Average precision" : average_precision}), ignore_index=True)

Best parameters for K-Nearest Neighbor:
k: 15
------------
Performance for K-Nearest Neighbor:
  - Accuracy score = 0.70
  - F1 score = 0.71
  - Precision score = 0.69
  - Recall score = 0.73
  - ROC AUC score = 0.77
  - Average precision score = 0.73


### 3.3. Model 3 : Support Vector Machines

In [11]:
# Choose parameter values to test for cross-validation
param_grid = {'C': [0.1, 1, 10, 100], 
              'gamma': [1, 0.1, 0.01, 0.001],
              'kernel': ['rbf']} 

# Choose the estimator
estimator = svm.SVC(probability=True)

# Cross-validation with GridSearchCV
clf = GridSearchCV(estimator, param_grid, scoring='f1', cv=StratifiedKFold(n_splits=3, shuffle=True))

# Fit data into the model
clf.fit(X_train, y_train)

# Predicting values
y_pred = clf.predict(X_test)

# Proba for the greater label
y_score = clf.predict_proba(X_test)[:, 1]

# Best parameters
best_param = clf.best_estimator_.get_params()
parameters = {'C':best_param['C'], 'gamma':best_param['gamma'], 'kernel':best_param['kernel']}
print('Best parameters for SVM:')
print('C:', parameters['C'])
print('gamma:', parameters['gamma'])
print('kernel:', parameters['kernel'])
print('------------')

# Calculate performance scores
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_score)
average_precision = average_precision_score(y_test, y_score)

# Print performance
print('Performance for SVM:')
print('  - Accuracy score = {:.2f}'.format(accuracy))
print('  - F1 score = {:.2f}'.format(f1))
print('  - Precision score = {:.2f}'.format(precision))
print('  - Recall score = {:.2f}'.format(recall))
print('  - ROC AUC score = {:.2f}'.format(roc_auc))
print('  - Average precision score = {:.2f}'.format(average_precision))

# Add performance to df_results
df_results = df_results.append(pd.Series({"Model" : "SVM",
                                          "Parameters": parameters,
                                          "Accuracy" : accuracy,
                                          "F1" : f1,
                                          "Precision": precision,
                                          "Recall" : recall,
                                          "ROC AUC" : roc_auc,
                                          "Average precision" : average_precision}), ignore_index=True)

Best parameters for SVM:
C: 0.1
gamma: 0.1
kernel: rbf
------------
Performance for SVM:
  - Accuracy score = 0.72
  - F1 score = 0.74
  - Precision score = 0.70
  - Recall score = 0.79
  - ROC AUC score = 0.80
  - Average precision score = 0.77


### 3.4. Model 4 : Multi-layer perceptrons

In [12]:
# Choose parameter values to test for cross-validation
param_grid = {'hidden_layer_sizes': np.arange(10, 20, 3),
              'activation': ['relu'],
              'solver': ['sgd', 'adam'],
              'alpha': [0.001, 0.05],
              'learning_rate': ['constant']}

# Choose the estimator
estimator = MLPClassifier(max_iter=300)

# Cross-validation with GridSearchCV
clf = GridSearchCV(estimator, param_grid, scoring='f1', cv=StratifiedKFold(n_splits=3, shuffle=True))

# Fit data into the model
clf.fit(X_train, y_train)

# Predicting values
y_pred = clf.predict(X_test)

# Proba for the greater label
y_score = clf.predict_proba(X_test)[:, 1]

# Best parameters
best_param = clf.best_estimator_.get_params()
parameters = {'hidden_layer_sizes':best_param['hidden_layer_sizes'], 'activation':best_param['activation'], 
              'solver':best_param['solver'], 'alpha':best_param['alpha'], 'learning_rate':best_param['learning_rate'], }
print('Best parameters for MLP:')
print('hidden_layer_sizes:', parameters['hidden_layer_sizes'])
print('activation:', parameters['activation'])
print('solver:', parameters['solver'])
print('alpha:', parameters['alpha'])
print('learning_rate:', parameters['learning_rate'])
print('------------')

# Calculate performance scores
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_score)
average_precision = average_precision_score(y_test, y_score)

# Print performance
print('Performance for MLP:')
print('  - Accuracy score = {:.2f}'.format(accuracy))
print('  - F1 score = {:.2f}'.format(f1))
print('  - Precision score = {:.2f}'.format(precision))
print('  - Recall score = {:.2f}'.format(recall))
print('  - ROC AUC score = {:.2f}'.format(roc_auc))
print('  - Average precision score = {:.2f}'.format(average_precision))

# Add performance to df_results
df_results = df_results.append(pd.Series({"Model" : "MLP",
                                          "Parameters": parameters,
                                          "Accuracy" : accuracy,
                                          "F1" : f1,
                                          "Precision": precision,
                                          "Recall" : recall,
                                          "ROC AUC" : roc_auc,
                                          "Average precision" : average_precision}), ignore_index=True)

Best parameters for MLP:
hidden_layer_sizes: 16
activation: relu
solver: adam
alpha: 0.05
learning_rate: constant
------------
Performance for MLP:
  - Accuracy score = 0.73
  - F1 score = 0.74
  - Precision score = 0.71
  - Recall score = 0.77
  - ROC AUC score = 0.81
  - Average precision score = 0.79


### 3.5. Model 5 : Quadratic discriminant analysis

In [13]:
# Choose parameter values to test for cross-validation
param_grid = [{'reg_param': [0.1, 0.2, 0.3, 0.4, 0.5]}]

# Choose the estimator
estimator = QuadraticDiscriminantAnalysis()

# Cross-validation with GridSearchCV
clf = GridSearchCV(estimator, param_grid, scoring='f1', cv=StratifiedKFold(n_splits=3, shuffle=True))

# Fit data into the model
clf.fit(X_train, y_train)

# Predicting values
y_pred = clf.predict(X_test)

# Proba for the greater label
y_score = clf.predict_proba(X_test)[:, 1]

# Best parameters
best_param = clf.best_estimator_.get_params()
parameters = {'reg_param':best_param['reg_param']}
print('Best parameters for Quadratic discriminant analysis:')
print('reg_param:', parameters['reg_param'])
print('------------')

# Calculate performance scores
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_score)
average_precision = average_precision_score(y_test, y_score)

# Print performance
print('Performance for Quadratic discriminant analysis:')
print('  - Accuracy score = {:.2f}'.format(accuracy))
print('  - F1 score = {:.2f}'.format(f1))
print('  - Precision score = {:.2f}'.format(precision))
print('  - Recall score = {:.2f}'.format(recall))
print('  - ROC AUC score = {:.2f}'.format(roc_auc))
print('  - Average precision score = {:.2f}'.format(average_precision))

# Add performance to df_results
df_results = df_results.append(pd.Series({"Model" : "Quadratic discriminant analysis",
                                          "Parameters": parameters,
                                          "Accuracy" : accuracy,
                                          "F1" : f1,
                                          "Precision": precision,
                                          "Recall" : recall,
                                          "ROC AUC" : roc_auc,
                                          "Average precision" : average_precision}), ignore_index=True)

Best parameters for Quadratic discriminant analysis:
reg_param: 0.5
------------
Performance for Quadratic discriminant analysis:
  - Accuracy score = 0.56
  - F1 score = 0.29
  - Precision score = 0.79
  - Recall score = 0.18
  - ROC AUC score = 0.77
  - Average precision score = 0.73


### 3.6. Model 6 : XGBoost

In [14]:
# Choose parameter values to test for cross-validation
param_grid = {'max_depth': range (2, 10, 1),
              'n_estimators': range(60, 220, 40),
              'learning_rate': [0.1, 0.01, 0.05]
}

# Choose the estimator
estimator = xgb.XGBClassifier(objective="binary:logistic", random_state=42)

# Cross-validation with GridSearchCV
clf = GridSearchCV(estimator, param_grid, scoring='f1', cv=StratifiedKFold(n_splits=3, shuffle=True))

# Fit data into the model
clf.fit(X_train, y_train)

# Predicting values
y_pred = clf.predict(X_test)

# Proba for the greater label
y_score = clf.predict_proba(X_test)[:, 1]

# Best parameters
best_param = clf.best_estimator_.get_params()
parameters = {'max_depth':best_param['max_depth'], 'n_estimators':best_param['n_estimators'], 'learning_rate':best_param['learning_rate']}
print('Best parameters for XGBoost:')
print('max_depth:', parameters['max_depth'])
print('n_estimators:', parameters['n_estimators'])
print('learning_rate:', parameters['learning_rate'])
print('------------')

# Calculate performance scores
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_score)
average_precision = average_precision_score(y_test, y_score)

# Print performance
print('Performance for XGBoost:')
print('  - Accuracy score = {:.2f}'.format(accuracy))
print('  - F1 score = {:.2f}'.format(f1))
print('  - Precision score = {:.2f}'.format(precision))
print('  - Recall score = {:.2f}'.format(recall))
print('  - ROC AUC score = {:.2f}'.format(roc_auc))
print('  - Average precision score = {:.2f}'.format(average_precision))

# Add performance to df_results
df_results = df_results.append(pd.Series({"Model" : "XGBoost",
                                          "Parameters": parameters,
                                          "Accuracy" : accuracy,
                                          "F1" : f1,
                                          "Precision": precision,
                                          "Recall" : recall,
                                          "ROC AUC" : roc_auc,
                                          "Average precision" : average_precision}), ignore_index=True)

Best parameters for XGBoost:
max_depth: 4
n_estimators: 100
learning_rate: 0.1
------------
Performance for XGBoost:
  - Accuracy score = 0.73
  - F1 score = 0.74
  - Precision score = 0.72
  - Recall score = 0.77
  - ROC AUC score = 0.81
  - Average precision score = 0.80


### 3.7. Models comparison

In [15]:
df_results.sort_values(by = "F1", axis=0, ascending=False).round(2).set_index('Model')

,Parameters,Accuracy,F1,Precision,Recall,ROC AUC,Average precision
Model,,,,,,,
XGBoost,"{'max_depth': 4, 'n_estimators': 100, 'learnin...",0.73,0.74,0.72,0.77,0.81,0.80
MLP,"{'hidden_layer_sizes': 16, 'activation': 'relu...",0.73,0.74,0.71,0.77,0.81,0.79
SVM,"{'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}",0.72,0.74,0.70,0.79,0.80,0.77
Logistic regression,"{'C': 1000.0, 'penalty': 'l2'}",0.71,0.72,0.70,0.74,0.79,0.76
K-Nearest Neighbor,{'k': 15},0.70,0.71,0.69,0.73,0.77,0.73
Quadratic discriminant analysis,{'reg_param': 0.5},0.56,0.29,0.79,0.18,0.77,0.73


## 4. Improve the best model (XGBoost) and investigate more

### 4.1. Improve XGBoost

In [88]:
# Cross-validation with XGBoost
# Choose parameter values to test for cross-validation
param_grid = {'max_depth': range (2, 10, 1),
              'n_estimators': range(60, 220, 40),
              'learning_rate': [0.1, 0.01, 0.05, 0.001],
              'reg_alpha': [1e-5, 1e-2, 0.1, 1, 100],
              'reg_lambda': [1e-5, 1e-2, 0.1, 1, 100],
              'gamma': [0, 0.1, 0.2, 0.3, 0.4],
              'booster': ['gbtree', 'gblinear']}

# Choose the estimator
estimator = xgb.XGBClassifier(objective="binary:logistic", random_state=42)

# Cross-validation with GridSearchCV
clf = GridSearchCV(estimator, param_grid, scoring='f1', cv=StratifiedKFold(n_splits=3, shuffle=True))

clf = HalvingRandomSearchCV(estimator, param_grid, scoring='f1', cv=StratifiedKFold(n_splits=2, shuffle=True, random_state=42))

# Fit data into the model
clf.fit(X_train, y_train)

# Predicting values
y_pred = clf.predict(X_test)

# Proba for the greater label
y_score = clf.predict_proba(X_test)[:, 1]

# Best parameters
best_param = clf.best_estimator_.get_params()
parameters = {'max_depth':best_param['max_depth'], 'n_estimators':best_param['n_estimators'], 'learning_rate':best_param['learning_rate'],
             'reg_alpha':best_param['reg_alpha'], 'reg_lambda':best_param['reg_lambda'], 'gamma':best_param['gamma'], 'booster':best_param['booster']}
print('Best parameters for XGBoost:')
if (parameters['booster']!='gblinear'):
    print('max_depth:', parameters['max_depth'])
    print('gamma:', parameters['gamma'])
print('n_estimators:', parameters['n_estimators'])
print('learning_rate:', parameters['learning_rate'])
print('reg_alpha:', parameters['reg_alpha'])
print('reg_lambda:', parameters['reg_lambda'])
print('booster:', parameters['booster'])
print('------------')

# Calculate performance scores
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_score)
average_precision = average_precision_score(y_test, y_score)

# Print performance
print('Performance for XGBoost:')
print('  - Accuracy score = {:.2f}'.format(accuracy))
print('  - F1 score = {:.2f}'.format(f1))
print('  - Precision score = {:.2f}'.format(precision))
print('  - Recall score = {:.2f}'.format(recall))
print('  - ROC AUC score = {:.2f}'.format(roc_auc))
print('  - Average precision score = {:.2f}'.format(average_precision))

[17:11:34] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "gamma", "max_depth" } are not used.

[17:11:35] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "gamma", "max_depth" } are not used.

[17:11:35] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "gamma", "max_depth" } are not used.

[17:11:35] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "gamma", "max_depth" } are not used.

[17:11:35] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: 

In [90]:
# See coeffs of the model
features = X_train.columns
if (parameters['booster']=='gblinear'):
    coefs = clf.best_estimator_.coef_
    features_XGboost = pd.DataFrame(coefs, index=features, columns=['coef'])
else:
    feature_importances = clf.best_estimator_.feature_importances_
    features_XGboost = pd.DataFrame(feature_importances, index=features, columns=['importance'])

features_XGboost.round(4)

,coef
HODECOD,0.0000
IETEST_Acute_Respiratory_Infection,0.0000
INCLAS_VACCINES,0.0000
SACAT_COMORBIDITIES,0.0000
SACAT_COMPLICATIONS,0.0000
SACAT_PREVIOUS_COVID-19_INFECTION,0.0000
MBTEST_OTHER RESPIRATORY PATHOGENS,0.0000
AGE,0.0425
LBTEST_APTT,0.0000
LBTEST_AST,0.0000


### 4.2. Test model for other continents

In [91]:
# Import data

df_asia = pd.read_csv(op.join(data_folder, 'df_asia.csv'), sep=',', index_col=0)
df_asia.name = 'df_asia'

#df_africa = pd.read_csv(op.join(data_folder, 'df_africa.csv'), sep=',', index_col=0)
#df_africa.name = 'df_africa'

df_south_america = pd.read_csv(op.join(data_folder, 'df_south_america.csv'), sep=',', index_col=0)
df_south_america.name = 'df_south_america'

df_north_america = pd.read_csv(op.join(data_folder, 'df_north_america.csv'), sep=',', index_col=0)
df_north_america.name = 'df_north_america'

df_oceania = pd.read_csv(op.join(data_folder, 'df_oceania.csv'), sep=',', index_col=0)
df_oceania.name = 'df_oceania'

In [92]:
# Do the list of the name of categorical columns
categorical = ['DSDECOD',
               'SEX',
               'CONTINENT_AF', 'CONTINENT_AS', 'CONTINENT_EU', 'CONTINENT_NA', 'CONTINENT_OC', 'CONTINENT_SA',
               'HODECOD', 
               'IETEST_Acute_Respiratory_Infection', 'IETEST_Cough', 'IETEST_Covid_ICU', 'IETEST_Covid_admission', 'IETEST_Dyspnoea_Tachypnoea', 'IETEST_Fever', 
               'IETEST_Inflammatory_MultiSystem_Syndrome', 'IETEST_noCovid_ICU',               
               'INCLAS_AGENTS_ACTING_ON_THE_RENIN-ANGIOTENSIN_SYSTEM', 'INCLAS_ANALGESICS', 'INCLAS_ANESTHETICS', 
               'INCLAS_ANTIBACTERIALS_FOR_SYSTEMIC_USE', 'INCLAS_ANTIHELMINTICS', 'INCLAS_ANTIINFLAMMATORY_AND_ANTIRHEUMATIC_PRODUCTS,_NON-STEROIDS',
               'INCLAS_ANTIMALARIALS', 'INCLAS_ANTIMYCOTICS_FOR_SYSTEMIC_USE', 'INCLAS_ANTITHROMBOTIC_AGENTS',
               'INCLAS_ANTIVIRALS_FOR_SYSTEMIC_USE', 'INCLAS_ARTIFICIAL_RESPIRATION', 'INCLAS_BETA_BLOCKING_AGENTS',
               'INCLAS_BLOOD_SUBSTITUTES_AND_PERFUSION_SOLUTIONS', 'INCLAS_BRONCHOSCOPY', 'INCLAS_CARDIAC_PACING',
               'INCLAS_CARDIAC_THERAPY', 'INCLAS_CARDIOPULMONARY_RESUSCITATION', 'INCLAS_CHEMOTHERAPY', 'INCLAS_CORTICOSTEROIDS_FOR_SYSTEMIC_USE',
               'INCLAS_DIURETICS', 'INCLAS_DRUGS_FOR_ACID_RELATED_DISORDERS', 'INCLAS_DRUGS_FOR_OBSTRUCTIVE_AIRWAY_DISEASES', 'INCLAS_EXTRACORPOREAL_MEMBRANE_OXYGENATION', 
               'INCLAS_HIGH_FLOW_OXYGEN_NASAL_CANNULA', 'INCLAS_IMMUNOGLOBULINS', 'INCLAS_IMMUNOSTIMULANTS', 'INCLAS_IMMUNOSUPPRESSANTS', 
               'INCLAS_INSERTION_OF_TRACHEOSTOMY_TUBE', 'INCLAS_INTUBATION', 'INCLAS_LIPID_MODIFYING_AGENTS', 'INCLAS_MUSCLE_RELAXANTS', 
               'INCLAS_NONINVASIVE_POSITIVE_PRESSURE_VENTILATION', 'INCLAS_NONINVASIVE_VENTILATION', 'INCLAS_OTHER_RESPIRATORY_SYSTEM_PRODUCTS', 
               'INCLAS_OXYGEN', 'INCLAS_PERCUTANEOUS_ENDOSCOPIC_GASTROSTOMY', 'INCLAS_PRONE_BODY_POSITION', 'INCLAS_PSYCHOLEPTICS', 
               'INCLAS_REMOVAL_OF_ENDOTRACHEAL_TUBE', 'INCLAS_RENAL_REPLACEMENT', 'INCLAS_REPLACEMENT_AGENT', 'INCLAS_TOTAL_PARENTERAL_NUTRITION', 
               'INCLAS_TRANSFUSION_OF_BLOOD_PRODUCT', 'INCLAS_VACCINES', 
               'MBTEST_ADENOVIRUS', 'MBTEST_BACTERIA','MBTEST_INFLUENZA', 'MBTEST_MB_SEVERE_ACUTE_RESP_SYND_CORONAVIRUS', 'MBTEST_OTHER PATHOGENS', 
               'MBTEST_OTHER RESPIRATORY PATHOGENS', 'MBTEST_RSV', 
               'RPSTRESC', 
               'RSCAT_AVPU', 
               'SACAT_COMORBIDITIES', 'SACAT_COMPLICATIONS', 'SACAT_PREVIOUS_COVID-19_INFECTION', 
               'SCTEST_Appropriate_Developmental_Age_Indicator', 'SCTEST_Breast_Fed_Indicator', 'SCTEST_Infant_Less_Than_One_Year_Indicator', 
               'SCTEST_Premature_Birth_Indicator']

In [93]:
# First for some columns in particular, it is more "logical" to put 0 (baseline) to fill NA

NA_to_0 = ['HODECOD', 
           'IETEST_Acute_Respiratory_Infection', 'IETEST_Cough', 'IETEST_Covid_ICU', 'IETEST_Covid_admission', 'IETEST_Dyspnoea_Tachypnoea', 'IETEST_Fever', 
           'IETEST_Inflammatory_MultiSystem_Syndrome', 'IETEST_noCovid_ICU',  
           'INCLAS_AGENTS_ACTING_ON_THE_RENIN-ANGIOTENSIN_SYSTEM', 'INCLAS_ANALGESICS', 'INCLAS_ANESTHETICS', 
           'INCLAS_ANTIBACTERIALS_FOR_SYSTEMIC_USE', 'INCLAS_ANTIHELMINTICS', 'INCLAS_ANTIINFLAMMATORY_AND_ANTIRHEUMATIC_PRODUCTS,_NON-STEROIDS',
           'INCLAS_ANTIMALARIALS', 'INCLAS_ANTIMYCOTICS_FOR_SYSTEMIC_USE', 'INCLAS_ANTITHROMBOTIC_AGENTS',
           'INCLAS_ANTIVIRALS_FOR_SYSTEMIC_USE', 'INCLAS_ARTIFICIAL_RESPIRATION', 'INCLAS_BETA_BLOCKING_AGENTS',
           'INCLAS_BLOOD_SUBSTITUTES_AND_PERFUSION_SOLUTIONS', 'INCLAS_BRONCHOSCOPY', 'INCLAS_CARDIAC_PACING',
           'INCLAS_CARDIAC_THERAPY', 'INCLAS_CARDIOPULMONARY_RESUSCITATION', 'INCLAS_CHEMOTHERAPY', 'INCLAS_CORTICOSTEROIDS_FOR_SYSTEMIC_USE',
           'INCLAS_DIURETICS', 'INCLAS_DRUGS_FOR_ACID_RELATED_DISORDERS', 'INCLAS_DRUGS_FOR_OBSTRUCTIVE_AIRWAY_DISEASES', 'INCLAS_EXTRACORPOREAL_MEMBRANE_OXYGENATION', 
           'INCLAS_HIGH_FLOW_OXYGEN_NASAL_CANNULA', 'INCLAS_IMMUNOGLOBULINS', 'INCLAS_IMMUNOSTIMULANTS', 'INCLAS_IMMUNOSUPPRESSANTS', 
           'INCLAS_INSERTION_OF_TRACHEOSTOMY_TUBE', 'INCLAS_INTUBATION', 'INCLAS_LIPID_MODIFYING_AGENTS', 'INCLAS_MUSCLE_RELAXANTS', 
           'INCLAS_NONINVASIVE_POSITIVE_PRESSURE_VENTILATION', 'INCLAS_NONINVASIVE_VENTILATION', 'INCLAS_OTHER_RESPIRATORY_SYSTEM_PRODUCTS', 
           'INCLAS_OXYGEN', 'INCLAS_PERCUTANEOUS_ENDOSCOPIC_GASTROSTOMY', 'INCLAS_PRONE_BODY_POSITION', 'INCLAS_PSYCHOLEPTICS', 
           'INCLAS_REMOVAL_OF_ENDOTRACHEAL_TUBE', 'INCLAS_RENAL_REPLACEMENT', 'INCLAS_REPLACEMENT_AGENT', 'INCLAS_TOTAL_PARENTERAL_NUTRITION', 
           'INCLAS_TRANSFUSION_OF_BLOOD_PRODUCT', 'INCLAS_VACCINES', 
           'MBTEST_ADENOVIRUS', 'MBTEST_BACTERIA','MBTEST_INFLUENZA', 'MBTEST_MB_SEVERE_ACUTE_RESP_SYND_CORONAVIRUS', 'MBTEST_OTHER PATHOGENS', 
           'MBTEST_OTHER RESPIRATORY PATHOGENS', 'MBTEST_RSV', 
           'RPSTRESC', 
           'RSCAT_AVPU', 
           'SACAT_COMORBIDITIES', 'SACAT_COMPLICATIONS']

In [94]:
# Preprocess new data

def process_continent(df_continent, categorical, NA_to_0):
    
     # 1. Replace NA by 0 in NA_to_0 columns
    
    df_continent[df_continent.columns[df_continent.columns.isin(NA_to_0)]] = df_continent[df_continent.columns[df_continent.columns.isin(NA_to_0)]].fillna(0)
    
    # 2. Separate in categorical and continuous columns
    
    df_continent_cat = df_continent[df_continent.columns[df_continent.columns.isin(categorical)]]
    df_continent_con = df_continent[df_continent.columns[~df_continent.columns.isin(categorical)]]
    
    cols_cat = df_continent_cat.columns
    cols_con = df_continent_con.columns
    
    # 3. Fill the missing values 
        
        # For categorical variables
    imp_cat = SimpleImputer(strategy = "most_frequent")
    imp_cat = imp_cat.fit(df_continent_cat)
    
    df_continent_cat = imp_cat.transform(df_continent_cat)
    df_continent_cat = pd.DataFrame(df_continent_cat, columns=cols_cat)
    
        # For continuous variables
    imp_con = SimpleImputer(strategy = "median")
    imp_con = imp_con.fit(df_continent_con)
    
    df_continent_con = imp_con.transform(df_continent_con)
    df_continent_con = pd.DataFrame(df_continent_con, columns=cols_con)

    # 4. Standardization of continuous data
    
    scaler = StandardScaler()
    scaler = scaler.fit(df_continent_con)
    
    df_continent_con = scaler.transform(df_continent_con)
    df_continent_con = pd.DataFrame(df_continent_con, columns=cols_con)
    
    # 5. Concat con and cat
    
    df_continent = pd.concat([df_continent_cat, df_continent_con], axis=1)
    
    return df_continent

In [104]:
# Create function to test model on another continent

def test_other_continents(df_continent, name_continent, clf):
    
    print(name_continent)
    print('')
    
    X_continent = df_continent.loc[:, df_continent.columns!='DSDECOD']
    y_continent = df_continent['DSDECOD']
    
    # Predicting values
    y_pred = clf.predict(X_continent)

    # Proba for the greater label
    y_score = clf.predict_proba(X_continent)[:, 1]

    # Best parameters
    best_param = clf.best_estimator_.get_params()
    parameters = {'max_depth':best_param['max_depth'], 'n_estimators':best_param['n_estimators'], 'learning_rate':best_param['learning_rate'],
                 'reg_alpha':best_param['reg_alpha'], 'reg_lambda':best_param['reg_lambda'], 'gamma':best_param['gamma'], 'booster':best_param['booster']}
    print('Best parameters for XGBoost trained on Europe:')
    if (parameters['booster']!='gblinear'):
        print('max_depth:', parameters['max_depth'])
        print('gamma:', parameters['gamma'])
    print('n_estimators:', parameters['n_estimators'])
    print('learning_rate:', parameters['learning_rate'])
    print('reg_alpha:', parameters['reg_alpha'])
    print('reg_lambda:', parameters['reg_lambda'])
    print('booster:', parameters['booster'])
    print('------------')

    # Calculate performance scores
    accuracy = accuracy_score(y_continent, y_pred)
    f1 = f1_score(y_continent, y_pred)
    precision = precision_score(y_continent, y_pred)
    recall = recall_score(y_continent, y_pred)
    roc_auc = roc_auc_score(y_continent, y_score)
    average_precision = average_precision_score(y_continent, y_score)
    
    # Print performance
    print('Performance for XGBoost trained on Europe:')
    print('  - Accuracy score = {:.2f}'.format(accuracy))
    print('  - F1 score = {:.2f}'.format(f1))
    print('  - Precision score = {:.2f}'.format(precision))
    print('  - Recall score = {:.2f}'.format(recall))
    print('  - ROC AUC score = {:.2f}'.format(roc_auc))
    print('  - Average precision score = {:.2f}'.format(average_precision))
    
    print('########################################')

In [105]:
# Test model for each continent 
continent_name = ['Asia', 'South America', 'North America', 'Oceania']
continent_list = [df_asia, df_south_america, df_north_america, df_oceania]
for df_continent, name_continent in zip(continent_list, continent_name):
    df_continent = process_continent(df_continent, categorical, NA_to_0)
    test_other_continents(df_continent, name_continent, clf)

C:\Users\felic\anaconda3\envs\ada\lib\site-packages\sklearn\impute\_base.py:49: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode = stats.mode(array)
C:\Users\felic\anaconda3\envs\ada\lib\site-packages\sklearn\impute\_base.py:49: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid thi

Asia

Best parameters for XGBoost trained on Europe:
n_estimators: 140
learning_rate: 0.001
reg_alpha: 0.1
reg_lambda: 1e-05
booster: gblinear
------------
Performance for XGBoost trained on Europe:
  - Accuracy score = 0.58
  - F1 score = 0.23
  - Precision score = 0.13
  - Recall score = 0.88
  - ROC AUC score = 0.77
  - Average precision score = 0.20
########################################
South America

Best parameters for XGBoost trained on Europe:
n_estimators: 140
learning_rate: 0.001
reg_alpha: 0.1
reg_lambda: 1e-05
booster: gblinear
------------
Performance for XGBoost trained on Europe:
  - Accuracy score = 0.55
  - F1 score = 0.31
  - Precision score = 0.19
  - Recall score = 0.81
  - ROC AUC score = 0.73
  - Average precision score = 0.27
########################################
North America

Best parameters for XGBoost trained on Europe:
n_estimators: 140
learning_rate: 0.001
reg_alpha: 0.1
reg_lambda: 1e-05
booster: gblinear
------------
Performance for XGBoost trained 

C:\Users\felic\anaconda3\envs\ada\lib\site-packages\sklearn\impute\_base.py:49: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode = stats.mode(array)
C:\Users\felic\anaconda3\envs\ada\lib\site-packages\sklearn\impute\_base.py:49: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid thi